In [16]:
GPT_config = {"vocab_size": 50257,
              "drop_rate" : 0.1,
              "context_length":1024,
              "n_heads": 12,
              "n_layers":12,
              "emb_dim":768,
              "qkv_bias":False
              
}

In [17]:
import torch
import torch.nn as nn

In [18]:
# torch.manual_seed(42)

# Gelu Activation function implementation for smoother Non linearities than Relu

In [19]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

# Layer Normalization class

In [20]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

# FeedForward Network

In [21]:
class Feedforward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'],4*cfg['emb_dim']),
            GELU(),
            nn.Linear(4*cfg['emb_dim'],cfg['emb_dim'])
        )
        
        
    def forward(self,x):
        return self.layers(x)

# Multihead Attention Block 

In [22]:
class MultiheadCausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout,n_heads,qkv_bias=False):
        super().__init__()
            
        assert (d_out % n_heads == 0), \
    "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = n_heads
        # self.head_dim = d_out // num_heads 
        self.head_dim = d_out // n_heads
        self.dropout = nn.Dropout(dropout)
        self.query_w = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.key_w = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.value_w = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
        
    def forward(self,x):
       b ,num_tokens, d_in= x.shape
       
       query = self.query_w(x).view(b, num_tokens, self.num_heads, self.head_dim)
       key = self.key_w(x).view(b, num_tokens, self.num_heads, self.head_dim)
       value = self.value_w(x).view(b, num_tokens, self.num_heads, self.head_dim)
        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
       key = key.transpose(1, 2)
       query = query.transpose(1, 2)
       value = value.transpose(1, 2)
       
       
       attn_scores = query @ key.transpose(2, 3)  # Dot product for each head
       
       
       mask_bool = self.mask[:num_tokens, :num_tokens].bool()
       attn_scores.masked_fill_(mask_bool, -torch.inf)
       attent_weights = torch.softmax(attn_scores/key.shape[-1] ** 0.5 , dim=-1)
       
       attent_weights = self.dropout(attent_weights)
       
       context_vec = (attent_weights @ value).transpose(1, 2) 
       
       context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
       
       context_vec = self.out_proj(context_vec)
       return context_vec
       
       

# Coding a Transformer Block 

In [23]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layer_norm1 =  LayerNorm(cfg['emb_dim'])
        self.layer_norm2 =  LayerNorm(cfg['emb_dim'])
        self.att = MultiheadCausalAttention(d_in=cfg['emb_dim'],d_out=cfg['emb_dim'],context_length=cfg['context_length'],n_heads=cfg['n_heads'],dropout=cfg['drop_rate'],qkv_bias=cfg['qkv_bias'])
        
        self.ff = Feedforward(cfg)
        self.shortcut_drop = nn.Dropout(cfg['drop_rate'])
        
        
    def forward(self,x):
        shortcut = x
        x = self.layer_norm1(x)
        x= self.att(x)
        x = self.shortcut_drop(x)
        x = x+ shortcut
        
        # The above x = x+ shortcut  is for adding the values from the input to the output of the dropout layer to avoid the vanishing Gradient problem during the back propagation.
        
        
        shortcut = x
        x = self.layer_norm2(x)
        x = self.ff(x)
        x = self.shortcut_drop(x)
        x = x+ shortcut
        return x

# This is the Gpt Block which will contain a 12 Layer NN architecture

In [24]:
class GptModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
        self.token_layer = nn.Embedding(cfg['vocab_size'],cfg['emb_dim'])
        self.pos_layer = nn.Embedding(cfg['context_length'],cfg['emb_dim'])
        
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        
        self.transformer_block = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )
        
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg['emb_dim'],cfg['vocab_size'], bias =False)
        
        
    def forward(self,x_id):
        batch_size, seq_len = x_id.shape
        
        tok_emb = self.token_layer(x_id)
        pos_emb = self.pos_layer(torch.arange(seq_len, device=x_id.device))
        
        x = tok_emb + pos_emb
        x = self.drop_emb(x)
        x = self.transformer_block(x)
        x = self.final_norm(x)

        logits = self.out_head(x)
        
        return logits        

In [25]:
GPT_config

{'vocab_size': 50257,
 'drop_rate': 0.1,
 'context_length': 1024,
 'n_heads': 12,
 'n_layers': 12,
 'emb_dim': 768,
 'qkv_bias': False}

## Define a batch input

In [26]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = "i love Machine Learning"
txt2 = "One step at "
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[   72,  1842, 10850, 18252],
        [ 3198,  2239,   379,   220]])


In [27]:
batch.shape

torch.Size([2, 4])

In [28]:
torch.manual_seed(123)
model = GptModel(GPT_config)
out = model(batch)
print("Input batch:\n", batch)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[   72,  1842, 10850, 18252],
        [ 3198,  2239,   379,   220]])

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.0424,  0.4464,  0.9319,  ...,  0.1077,  1.0280,  0.3325],
         [ 0.6158,  0.2995, -0.3806,  ..., -0.2127, -0.4360,  0.5152],
         [ 0.6728,  0.3157, -0.3430,  ...,  0.3751, -0.7065,  0.1711],
         [-0.2579,  0.0941, -0.4393,  ...,  1.1880, -0.3813, -0.2190]],

        [[ 0.0318, -0.5157,  0.1368,  ..., -0.2589, -0.5309, -0.5686],
         [ 0.7093, -0.1975, -0.6394,  ..., -0.2053, -0.4661,  0.6504],
         [ 0.1872, -0.1331,  0.3191,  ...,  0.1674, -0.0223, -0.1469],
         [ 0.0863, -0.4963, -0.0773,  ...,  0.4304, -1.1178,  0.1237]]],
       grad_fn=<UnsafeViewBackward0>)


In [29]:
total = sum([p.numel() for p in model.parameters() if p.requires_grad==True])

In [30]:
total - sum(p.numel() for p in model.out_head.parameters())

124412160

<function torch._VariableFunctionsClass.empty>